### Final simulation for testing the best model on a batch of increasing 100 samples in the train set and test on the next 100 batch

In [1]:
# Importing the required libraries
import numpy as np
import os
import pandas as pd
from ast import literal_eval

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB

# Libraries for xgboost,light gradient boosting and Cat boosting
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# We can build some calibaration curve
from sklearn.calibration import calibration_curve

import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt

import random
# Chaning back to the resource directory if it is not there
if("Res" not in os.getcwd()):
    os.chdir("..")

/Users/gouravsanjuktabhabesh/.conda/envs/ml_101_env/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Reading the input file for this notebook called attributes_df.csv containing the attributes of each mail
# extracted earlier
attributes_hot_encode = pd.read_csv("Res/processed_data/attributes_df.csv",sep=",")

# Removing those mails that have no text inside them such as Res/enron1/spam/1733.2004-07-29.GP.spam.txt
attributes_hot_encode.drop(attributes_hot_encode.index\
                           [np.where(np.isnan(attributes_hot_encode[["median_useful_token_len"]]))[0]],inplace=True)
col_attributes = []
with open("Res/processed_data/attributes_names.txt", 'r') as f:
    col_attributes.extend(f.read().split())

In [4]:
# Change this value from "enron1" to "enron6" to build all the simulations
enron_dataset_list = ["enron1","enron2","enron3","enron4","enron5","enron6"]
for enron_dataset in enron_dataset_list:
    print("Running simulation for "+enron_dataset+"...")
    attributes_hot_encode_ordered = attributes_hot_encode.loc[attributes_hot_encode["dataset"]==enron_dataset].sort_values("date_mail")
    attributes_hot_encode_1_spam = attributes_hot_encode_ordered.loc[(attributes_hot_encode_ordered["dataset"]==enron_dataset) &
                                                             (attributes_hot_encode_ordered["mail_label"]=="spam")]
    attributes_hot_encode_1_ham = attributes_hot_encode_ordered.loc[(attributes_hot_encode_ordered["dataset"]==enron_dataset) &
                                                             (attributes_hot_encode_ordered["mail_label"]=="ham")]

    attributes_hot_encoded_df = pd.DataFrame(columns = list(attributes_hot_encode_ordered))
    i = 0
    if(len(attributes_hot_encode_1_ham)>len(attributes_hot_encode_1_spam)):
        for j in range(len(attributes_hot_encode_1_spam)):
            ham_count = np.random.choice([0,1,2,3,4,5,6,7],p=[0.025,0.15,0.25,0.25,0.15,0.10,0.05,0.025])
            try:
                attributes_hot_encoded_df = pd.concat([attributes_hot_encoded_df,
                                                       attributes_hot_encode_1_spam.iloc[j:j+1],
                                                       attributes_hot_encode_1_ham.iloc[i:i+ham_count]])
                i+=ham_count
                if(j==len(attributes_hot_encode_1_spam)-1):
                    attributes_hot_encoded_df = pd.concat([attributes_hot_encoded_df,
                                                           attributes_hot_encode_1_ham.iloc[i:]])
            except:
                attributes_hot_encoded_df = pd.concat([attributes_hot_encoded_df,
                                                       attributes_hot_encode_1_spam.iloc[j:]])
                break
    else:
        for j in range(len(attributes_hot_encode_1_ham)):
            spam_count = np.random.choice([0,1,2,3,4,5,6,7],p=[0.025,0.15,0.25,0.25,0.15,0.10,0.05,0.025])
            try:
                attributes_hot_encoded_df = pd.concat([attributes_hot_encoded_df,
                                                       attributes_hot_encode_1_ham.iloc[j:j+1],
                                                       attributes_hot_encode_1_spam.iloc[i:i+spam_count]])
                i+=spam_count
                if(j==len(attributes_hot_encode_1_ham)-1):
                    attributes_hot_encoded_df = pd.concat([attributes_hot_encoded_df,
                                                           attributes_hot_encode_1_spam.iloc[i:]])
            except:
                attributes_hot_encoded_df = pd.concat([attributes_hot_encoded_df,
                                                       attributes_hot_encode_1_ham.iloc[j:]])
                break
    f = list(attributes_hot_encoded_df["mail_label"])
    train_df = pd.DataFrame(columns=list(attributes_hot_encoded_df))
    measure_after_iteration = pd.DataFrame(columns = ["Train_spam_count","Train_ham_count","Test_spam_count",
                                                      "Test_ham_count","True_negative","False_positive",
                                                      "False_negative","True_positive","ratio_per_batch"])
    for i in range((len(f)/100)-1):
        train_df = pd.concat([train_df,attributes_hot_encoded_df.iloc[i*100:((i+1)*100)]])
        if(i==(len(f)/100)-2):
            test_df = attributes_hot_encoded_df.iloc[(i+1)*100:]
        else:
            test_df = attributes_hot_encoded_df.iloc[(i+1)*100:((i+2)*100)]
        print("train = {}, test = {}".format(len(train_df),len(test_df)))

        baseline_X_features = ['line_count', 'token_count', 'punctuations_count', 'single_char_count', 
                             'number_token_count', 'year_count', 'stopword_count','median_useful_token_len', 
                             'avg_useful_token_len', 'rareword_count','attributes_len']
        for col in baseline_X_features:
            train_df[col] = train_df[col].astype(float)
            test_df[col] = test_df[col].astype(float)
        baseline_X_features.extend(col_attributes)
        Y_feature = "mail_label"

        X_train = train_df[baseline_X_features]
        y_train = train_df[Y_feature]

        X_test = test_df[baseline_X_features]
        y_test = test_df[Y_feature]

        y_train["label"] = y_train.apply(lambda x: 1.0 if x=="spam" else 0.0)
        y_test["label"] = y_test.apply(lambda x: 1.0 if x=="spam" else 0.0)

        dtrain = xgb.DMatrix(X_train, label=y_train["label"])
        dtest = xgb.DMatrix(X_test, label=y_test["label"])

        param = {'max_depth': 100, 'eta': 1, 'verbosity': 0, 'objective': 'binary:logistic',"n_estimators": [200]}
        param['nthread'] = 4
        param['eval_metric'] =  ['auc']

        evallist = [(dtest, 'eval'), (dtrain, 'train')]

        num_round = 30
        try:
            bst = xgb.train(param, dtrain, num_round, evallist)
        except:
            break
        y_pred = bst.predict(dtest)
        y_pred = pd.Series(y_pred).apply(lambda x: 1.0 if x>0.5 else 0.0)

        tn, fp, fn, tp = confusion_matrix(y_test["label"], y_pred).ravel()
        print("True Positives(spam) = {}".format(tp))
        print("True Negatives(ham)  = {}".format(tn))
        print("False Positives(actual=ham, predicted=spam) = {}".format(fp))
        print("False Negatives(actual=spam,predicted=ham) = {}".format(fn))
        print("Accuracy = {:.2f}%".format((tp+tn)/float(len(y_test))*100))
        print("Recall = {:.2f}%".format((tp/float(tp+fn))*100))
        print("Specificity = {:.2f}%".format((tn/float(tn+fp))*100))
        test_ham_count = len(test_df.loc[test_df["mail_label"]=="ham"])
        test_spam_count = len(test_df.loc[test_df["mail_label"]=="spam"])
        measure_after_iteration.loc[i] = [len(train_df.loc[train_df["mail_label"]=="spam"]),\
                                          len(train_df.loc[train_df["mail_label"]=="ham"]),\
                                          test_spam_count,test_ham_count,tn, fp, fn, tp,
                                          float(test_ham_count)/test_spam_count]
        
    measure_after_iteration["Recall"] = ((measure_after_iteration["True_positive"]/(measure_after_iteration["True_positive"]+
                                                                    measure_after_iteration["False_negative"]))*100.0)
    measure_after_iteration["Specificity"] = ((measure_after_iteration["True_negative"]/(measure_after_iteration["True_negative"]+
                                                                            measure_after_iteration["False_positive"]))*100.0)

    measure_after_iteration.to_csv("Res/processed_data/measure_after_iteration_"+enron_dataset+".csv",sep=",",index=False)
    attributes_hot_encoded_df.to_csv("Res/processed_data/attributes_hot_encoded_df_"+enron_dataset+".csv",sep=",",index=False)

    # Plotting the graphs for each simulation
    plt.close()
    plt.plot(range(2,len(measure_after_iteration)+1),measure_after_iteration["Recall"][1:],"-or")
    plt.plot(range(2,len(measure_after_iteration)+1),measure_after_iteration["Specificity"][1:],"-ob")
    plt.xlabel("Number of emails X 100")
    plt.legend(("Spam Recall","Ham Recall"),loc='lower right')
    plt.title(enron_dataset)
    plt.grid()
    plt.savefig("Res/processed_data/plot_simulation_"+enron_dataset+".jpg")

    plt.close()
    plt.plot(range(2,len(measure_after_iteration)+1),measure_after_iteration["ratio_per_batch"][1:],"-og")
    plt.xlabel("Batch Number")
    plt.title(enron_dataset+" - ham:spam ratio per batch")
    plt.grid()
    plt.savefig("Res/processed_data/plot_simulation_"+enron_dataset+"_fluctuation.jpg")
    print("Done !")

Running simulation for enron3...
train = 100, test = 100


/Users/gouravsanjuktabhabesh/.conda/envs/ml_101_env/lib/python2.7/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0]	eval-auc:0.701143	train-auc:0.983279


/Users/gouravsanjuktabhabesh/.conda/envs/ml_101_env/lib/python2.7/site-packages/ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/gouravsanjuktabhabesh/.conda/envs/ml_101_env/lib/python2.7/site-packages/ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[1]	eval-auc:0.852911	train-auc:0.998904
[2]	eval-auc:0.901507	train-auc:1
[3]	eval-auc:0.942308	train-auc:1
[4]	eval-auc:0.956341	train-auc:1
[5]	eval-auc:0.955821	train-auc:1
[6]	eval-auc:0.949584	train-auc:1
[7]	eval-auc:0.948025	train-auc:1
[8]	eval-auc:0.954262	train-auc:1
[9]	eval-auc:0.948025	train-auc:1
[10]	eval-auc:0.943867	train-auc:1
[11]	eval-auc:0.945946	train-auc:1
[12]	eval-auc:0.945426	train-auc:1
[13]	eval-auc:0.948025	train-auc:1
[14]	eval-auc:0.950104	train-auc:1
[15]	eval-auc:0.950624	train-auc:1
[16]	eval-auc:0.949584	train-auc:1
[17]	eval-auc:0.952183	train-auc:1
[18]	eval-auc:0.951143	train-auc:1
[19]	eval-auc:0.952703	train-auc:1
[20]	eval-auc:0.949584	train-auc:1
[21]	eval-auc:0.949584	train-auc:1
[22]	eval-auc:0.952183	train-auc:1
[23]	eval-auc:0.950104	train-auc:1
[24]	eval-auc:0.951663	train-auc:1
[25]	eval-auc:0.952703	train-auc:1
[26]	eval-auc:0.952703	train-auc:1
[27]	eval-auc:0.952703	train-auc:1
[28]	eval-auc:0.953742	train-auc:1
[29]	eval-auc:0.954782

...

[20]	eval-auc:1	train-auc:1
[21]	eval-auc:1	train-auc:1
[22]	eval-auc:1	train-auc:1
[23]	eval-auc:1	train-auc:1
[24]	eval-auc:1	train-auc:1
[25]	eval-auc:1	train-auc:1
[26]	eval-auc:1	train-auc:1
[27]	eval-auc:1	train-auc:1
[28]	eval-auc:1	train-auc:1
[29]	eval-auc:1	train-auc:1
True Positives(spam) = 77
True Negatives(ham)  = 22
False Positives(actual=ham, predicted=spam) = 1
False Negatives(actual=spam,predicted=ham) = 0
Accuracy = 98.02%
Recall = 100.00%
Specificity = 95.65%
train = 2800, test = 100
[0]	eval-auc:0.939878	train-auc:0.998113
[1]	eval-auc:0.987316	train-auc:0.999671
[2]	eval-auc:0.986809	train-auc:0.999903
[3]	eval-auc:0.989345	train-auc:0.99998
[4]	eval-auc:0.99036	train-auc:0.999997
[5]	eval-auc:0.987823	train-auc:1
[6]	eval-auc:0.984779	train-auc:1
[7]	eval-auc:0.977676	train-auc:1
[8]	eval-auc:0.974632	train-auc:1
[9]	eval-auc:0.967529	train-auc:1
[10]	eval-auc:0.975139	train-auc:1
[11]	eval-auc:0.977169	train-auc:1
[12]	eval-auc:0.979198	train-auc:1
[13]	eval-auc:

[3]	eval-auc:0.977676	train-auc:0.999999
[4]	eval-auc:0.976662	train-auc:1
[5]	eval-auc:0.967022	train-auc:1
[6]	eval-auc:0.972603	train-auc:1
[7]	eval-auc:0.98072	train-auc:1
[8]	eval-auc:0.984779	train-auc:1
[9]	eval-auc:0.982243	train-auc:1
[10]	eval-auc:0.984272	train-auc:1
[11]	eval-auc:0.985287	train-auc:1
[12]	eval-auc:0.983765	train-auc:1
[13]	eval-auc:0.985287	train-auc:1
[14]	eval-auc:0.986301	train-auc:1
[15]	eval-auc:0.986301	train-auc:1
[16]	eval-auc:0.985287	train-auc:1
[17]	eval-auc:0.987316	train-auc:1
[18]	eval-auc:0.987823	train-auc:1
[19]	eval-auc:0.989345	train-auc:1
[20]	eval-auc:0.988331	train-auc:1
[21]	eval-auc:0.990868	train-auc:1
[22]	eval-auc:0.991882	train-auc:1
[23]	eval-auc:0.99239	train-auc:1
[24]	eval-auc:0.993404	train-auc:1
[25]	eval-auc:0.993404	train-auc:1
[26]	eval-auc:0.993912	train-auc:1
[27]	eval-auc:0.994419	train-auc:1
[28]	eval-auc:0.994926	train-auc:1
[29]	eval-auc:0.994419	train-auc:1
True Positives(spam) = 73
True Negatives(ham)  = 22
False

[18]	eval-auc:1	train-auc:1
[19]	eval-auc:1	train-auc:1
[20]	eval-auc:1	train-auc:1
[21]	eval-auc:1	train-auc:1
[22]	eval-auc:1	train-auc:1
[23]	eval-auc:1	train-auc:1
[24]	eval-auc:1	train-auc:1
[25]	eval-auc:1	train-auc:1
[26]	eval-auc:1	train-auc:1
[27]	eval-auc:1	train-auc:1
[28]	eval-auc:1	train-auc:1
[29]	eval-auc:1	train-auc:1
True Positives(spam) = 73
True Negatives(ham)  = 26
False Positives(actual=ham, predicted=spam) = 1
False Negatives(actual=spam,predicted=ham) = 0
Accuracy = 98.02%
Recall = 100.00%
Specificity = 96.30%
train = 4100, test = 100
[0]	eval-auc:0.972332	train-auc:0.998038
[1]	eval-auc:0.985319	train-auc:0.999839
[2]	eval-auc:0.990966	train-auc:0.999971
[3]	eval-auc:0.998871	train-auc:1
[4]	eval-auc:0.995483	train-auc:0.999999
[5]	eval-auc:0.997177	train-auc:1
[6]	eval-auc:0.998306	train-auc:1
[7]	eval-auc:0.998871	train-auc:1
[8]	eval-auc:0.998871	train-auc:1
[9]	eval-auc:0.998871	train-auc:1
[10]	eval-auc:1	train-auc:1
[11]	eval-auc:1	train-auc:1
[12]	eval-au

[10]	eval-auc:0.996711	train-auc:1
[11]	eval-auc:0.996711	train-auc:1
[12]	eval-auc:0.995066	train-auc:1
[13]	eval-auc:0.995614	train-auc:1
[14]	eval-auc:0.996711	train-auc:1
[15]	eval-auc:0.996711	train-auc:1
[16]	eval-auc:0.996711	train-auc:1
[17]	eval-auc:0.996162	train-auc:1
[18]	eval-auc:0.996711	train-auc:1
[19]	eval-auc:0.996711	train-auc:1
[20]	eval-auc:0.996711	train-auc:1
[21]	eval-auc:0.996711	train-auc:1
[22]	eval-auc:0.997259	train-auc:1
[23]	eval-auc:0.997259	train-auc:1
[24]	eval-auc:0.997259	train-auc:1
[25]	eval-auc:0.997259	train-auc:1
[26]	eval-auc:0.997259	train-auc:1
[27]	eval-auc:0.997259	train-auc:1
[28]	eval-auc:0.996711	train-auc:1
[29]	eval-auc:0.996711	train-auc:1
True Positives(spam) = 73
True Negatives(ham)  = 22
False Positives(actual=ham, predicted=spam) = 2
False Negatives(actual=spam,predicted=ham) = 3
Accuracy = 94.06%
Recall = 96.05%
Specificity = 91.67%
train = 4800, test = 100
[0]	eval-auc:0.997288	train-auc:0.998276
[1]	eval-auc:0.998794	train-auc:

[28]	eval-auc:0.998985	train-auc:1
[29]	eval-auc:0.998985	train-auc:1
True Positives(spam) = 73
True Negatives(ham)  = 24
False Positives(actual=ham, predicted=spam) = 3
False Negatives(actual=spam,predicted=ham) = 0
Accuracy = 96.04%
Recall = 100.00%
Specificity = 88.89%
train = 5400, test = 100
[0]	eval-auc:0.921518	train-auc:0.998257
[1]	eval-auc:0.983368	train-auc:0.999797
[2]	eval-auc:0.991684	train-auc:0.999978
[3]	eval-auc:0.994802	train-auc:0.999991
[4]	eval-auc:0.997401	train-auc:1
[5]	eval-auc:0.997921	train-auc:1
[6]	eval-auc:0.998441	train-auc:1
[7]	eval-auc:0.998441	train-auc:1
[8]	eval-auc:0.998441	train-auc:1
[9]	eval-auc:0.998441	train-auc:1
[10]	eval-auc:0.99896	train-auc:1
[11]	eval-auc:0.998441	train-auc:1
[12]	eval-auc:0.998441	train-auc:1
[13]	eval-auc:0.998441	train-auc:1
[14]	eval-auc:0.99896	train-auc:1
[15]	eval-auc:0.99896	train-auc:1
[16]	eval-auc:0.99896	train-auc:1
[17]	eval-auc:0.99896	train-auc:1
[18]	eval-auc:0.99896	train-auc:1
[19]	eval-auc:0.99896	tra

In [8]:
# Run when all the 6 enron datasets have been made to run
# Average of all the batches for each enron dataset
for i in range(6):
    enron_dataset="enron"+str(i+1)
    measure_after_iteration = pd.read_csv("Res/processed_data/measure_after_iteration_"+enron_dataset+".csv",sep=",")
    print(enron_dataset,np.round(np.mean(measure_after_iteration["Recall"]),1),np.round(np.mean(measure_after_iteration["Specificity"]),1))

('enron1', 88.1, 97.6)
('enron2', 89.8, 97.7)
('enron3', 91.9, 98.2)
('enron4', 98.5, 93.5)
('enron5', 98.9, 91.2)
('enron6', 98.8, 89.1)


In [6]:
print("Completed !")

Completed !
